In [ ]:
# !pip install langchain cohere
# !pip install langchain_community
# !pip install langchain_cohere

In [244]:
import cohere
from langchain_cohere import ChatCohere

In [261]:
cohere_api_key = 'API_COHERE'

llm = ChatCohere(
    cohere_api_key=cohere_api_key, 
    model='command-r-plus', 
    temprature=0.1
)

In [262]:
system_message = ("""
شما به عنوان یک دستیار هوشمند بانکی عمل می‌کنید.

وظیفه شما این است که نیت کاربر را تشخیص داده و اطلاعات مورد نیاز (اسلات‌ها) را استخراج کنید.

پاسخ را به صورت دقیق و در قالب زیر ارائه دهید:

نیت: [نیت کاربر]
اسلات‌ها: [اسلات‌ها به صورت دیکشنری پایتون]

اگر نیت یا اسلات‌ها را نمی‌توانید تشخیص دهید، بنویسید:

نیت: نامشخص
اسلات‌ها: '{{{{}}}}'

مثال‌ها:

کاربر: "موجودی حسابم چقدره؟"
پاسخ:
نیت: نمایش موجودی
اسلات‌ها: '{{{{}}}}'

کاربر: "می‌خوام ۵۰۰,۰۰۰ ریال به حساب 1234567890 منتقل کنم."
پاسخ:
نیت: انتقال وجه
اسلات‌ها: '{{{{'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '1234567890'}}}}'

کاربر: "اپلیکیشن بانک همراه کار نمی‌کنه."
پاسخ:
نیت: رفع مشکل
اسلات‌ها: '{{{{'شرح مشکل': 'اپلیکیشن بانک همراه کار نمی‌کند'}}}}'

غیر از این خروجی هیچ خروجی دیگری نده
""")

In [263]:
examples = [
    {
        "question": "موجودی حسابم چقدره؟",
        "intent": "نمایش موجودی",
        "slots": "{}"
    },
    {
        "question": "می‌خوام ۵۰۰,۰۰۰ ریال به حساب 1234567890 منتقل کنم.",
        "intent": "انتقال وجه",
        "slots": "{'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '1234567890'}"
    },
    {
        "question": "آخرین تراکنش‌هامو می‌خوام ببینم.",
        "intent": "نمایش تاریخچه تراکنش‌ها",
        "slots": "{}"
    },
    {
        "question": "اپلیکیشن بانک همراه کار نمی‌کنه.",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'اپلیکیشن بانک همراه کار نمی‌کند'}"
    },
    {
        "question": "چطور می‌تونم پول انتقال بدم؟",
        "intent": "انتقال وجه",
        "slots": "{}"
    },
    {
        "question": "مشکل ورود به حساب دارم.",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'مشکل در ورود به حساب'}"
    },
    {
        "question": "میرم داخل پروفایلم عکسمو نشون نمیده",
        "intent": "رفع مشکل",
        "slots": "{'شرح مشکل': 'مشکل در دیدن عکس پروفایل'}"
    },
]


In [264]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "نیت: {intent}\nاسلات‌ها: {slots}")
    ]
)

In [265]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [266]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    few_shot_prompt,
    ("human", "{question}")
])

In [267]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

In [277]:
def show_balance():
    #درخواست به دیتابیس برای گرفتن اطلاعات
    return "موجودی حساب شما ۱۰,۰۰۰,۰۰۰ ریال است."

def transaction_history():
    #درخواست به دیتابیس برای گرفتن اطلاعات تراکنش ها
    return {
        "تراکنش ها": [
            {"تاریخ": "2024-11-01", "مقدار": "500,000", "type": "برداشت"},
            {"تاریخ": "2024-10-25", "مقداار": "1,000,000", "type": "واریز"}
        ]
    }

In [281]:
def print_output(response):
    if "تراکنش" in response['text']:
        print(transaction_history())
    elif "موجودی" in response['text']:
        print(show_balance())
    else:
        print(response['text'])

# مثال اول

In [275]:
prompt = "تاریخچه تراکنش هام چیه"
response = chain.invoke({"question":prompt})

In [282]:
print_output(response)

{'تراکنش ها': [{'تاریخ': '2024-11-01', 'مقدار': '500,000', 'type': 'برداشت'}, {'تاریخ': '2024-10-25', 'مقداار': '1,000,000', 'type': 'واریز'}]}


# مثال دوم

In [283]:
prompt = "عکس پروفایلم مشکل داره باز نمیشه"
response = chain.invoke({"question":prompt})

In [284]:
print_output(response)

نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'عکس پروفایل باز نمی‌شود'}


# مثال سوم

In [287]:
prompt = "وقتی میرم توی پروفایلم داراییم درست نیست"
response = chain.invoke({"question":prompt})

In [288]:
print_output(response)

نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'مشکل در نمایش دارایی در پروفایل'}


# مثال چهارم

In [289]:
prompt = "وقتی کلیک میکنم هچی واسم نشون نمیده"
response = chain.invoke({"question":prompt})

In [290]:
print_output(response)

نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'هنگ کردن اپلیکیشن'}


# مثال پنجم

In [295]:
prompt = "میتونی ۵۰۰,۰۰۰ ریال بزن ب حساب 9876543210"
response = chain.invoke({"question":prompt})

In [296]:
print_output(response)

نیت: انتقال وجه
اسلات‌ها: {'مبلغ': '۵۰۰,۰۰۰ ریال', 'شماره حساب مقصد': '9876543210'}


# مثال ششم

In [293]:
prompt = "یک میلیون تومن بزن ب حساب 1234567890"
response = chain.invoke({"question":prompt})

In [294]:
print_output(response)

نیت: انتقال وجه
اسلات‌ها: {'مبلغ': 'یک میلیون تومان', 'شماره حساب مقصد': '1234567890'}


# مثال هفتم

In [297]:
prompt = "میخوام یک میلیون تومن انتقال بدم ب حساب 1234567890 خطا میگیرم موقع انتقال"
response = chain.invoke({"question":prompt})

In [298]:
print_output(response)

نیت: رفع مشکل
اسلات‌ها: {'شرح مشکل': 'خطا در هنگام انتقال وجه', 'مبلغ': 'یک میلیون تومان', 'شماره حساب مقصد': '1234567890'}
